In [1]:
import pandas as pd

import psycopg2 as pg

from sqlalchemy import create_engine, Table, Column, MetaData, DATE, VARCHAR

## Conectando com o banco de dados

In [2]:
# Configurações da conexão
usuario = 'postgres'
senha = '123,258D'
host = 'localhost'
porta = '5432'
banco_de_dados = 'my_test'

# URL de conexão
url = f"postgresql://{usuario}:{senha}@{host}:{porta}/{banco_de_dados}"

# Criar o engine
engine = create_engine(url)

# Teste a conexão
try:
    connection = engine.connect()
    print("Conexão bem-sucedida!")
except Exception as e:
    print(f"Erro ao conectar: {e}")
finally:
    connection.close()


Conexão bem-sucedida!


## Inserindo arquivo csv no banco de dados conectado

In [3]:
arquivo = '/home/ferdanielinux/desktop-linux/scripts-python/proj-aries/Dados/CNES/tbEstabelecimento202406.csv'

In [4]:
df = pd.read_csv(arquivo, sep=';', encoding = 'latin-1')

/tmp/ipykernel_1124/4038384760.py:1: DtypeWarning: Columns (0,14,15,36,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(arquivo, sep=';', encoding = 'latin-1')


In [5]:
print("Número de linhas: ", df.shape[0])
print("Número de colunas: ", df.shape[1])

Número de linhas:  535727
Número de colunas:  55


Acertando os tipos dos dados conforme o dicionário encontrado em: https://cnes.datasus.gov.br/pages/downloads/documentacao.jsp -> Dicionário de Dados do SCNES

Atributos de interesse:
CO_UNIDADE, CO_CNES, NU_CNPJ_MANTENEDORA, NO_RAZAO_SOCIAL, NO_FANTASIA, NO_LOGRADOURO, NU_ENDERECO, NO_COMPLEMENTO, NO_BAIRRO, CO_CEP, CO_REGIAO_SAUDE, NU_TELEFONE, NO_EMAIL e DT_EXPEDICAO

In [6]:
metadata = MetaData()

table_CNES = Table('table_CNES', metadata,
                Column('CO_UNIDADE', VARCHAR(31), primary_key = True, nullable = False),
                Column('CO_CNES', VARCHAR(7)),
                Column('NU_CNPJ_MANTENEDORA', VARCHAR(16)),
                Column('NO_RAZAO_SOCIAL', VARCHAR(60), nullable = False),
                Column('NO_FANTASIA', VARCHAR(60), nullable = False),
                Column('NO_LOGRADOURO', VARCHAR(60), nullable = False),
                Column('NU_ENDERECO', VARCHAR(10)),
                Column('NO_COMPLEMENTO', VARCHAR(20)),
                Column('NO_BAIRRO', VARCHAR(20), nullable = False),
                Column('CO_CEP', VARCHAR(8), nullable = False),
                Column('CO_REGIAO_SAUDE', VARCHAR(4)),
                Column('NU_TELEFONE', VARCHAR(33)),
                Column('NO_EMAIL', VARCHAR(60)),
                Column('DT_EXPEDICAO', VARCHAR(20))
)

In [7]:
metadata.create_all(engine)

In [ ]:
df2 = df.filter(items = table_CNES.columns.keys())

df2.head(7)

In [12]:
# Calcula o tamanho máximo de cada coluna no dataframe
max_lengths = df2.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0).max()

print(max_lengths)

/tmp/ipykernel_1124/2998215887.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  max_lengths = df2.applymap(lambda x: len(str(x)) if pd.notnull(x) else 0).max()


CO_UNIDADE             31
CO_CNES                 7
NU_CNPJ_MANTENEDORA    16
NO_RAZAO_SOCIAL        60
NO_FANTASIA            60
NO_LOGRADOURO          60
NU_ENDERECO            10
NO_COMPLEMENTO         20
NO_BAIRRO              20
CO_CEP                  8
CO_REGIAO_SAUDE         4
NU_TELEFONE            33
NO_EMAIL               60
DT_EXPEDICAO           20
dtype: int64


Colocando todos os dados presentes no df2 no banco de dados criado

In [13]:
# Inserir apenas os dez primeiros dados no banco de dados
df2.head(500).to_sql('table_CNES', engine, if_exists = 'replace', index = False, dtype = {
    'CO_UNIDADE': VARCHAR(31),
    'CO_CNES': VARCHAR(7),
    'NU_CNPJ_MANTENEDORA': VARCHAR(16),
    'NO_RAZAO_SOCIAL': VARCHAR(60),
    'NO_FANTASIA': VARCHAR(60),
    'NO_LOGRADOURO': VARCHAR(60),
    'NU_ENDERECO': VARCHAR(10),
    'NO_COMPLEMENTO': VARCHAR(20),
    'NO_BAIRRO': VARCHAR(20),
    'CO_CEP': VARCHAR(8),
    'CO_REGIAO_SAUDE': VARCHAR(4),
    'NU_TELEFONE': VARCHAR(33),
    'NO_EMAIL': VARCHAR(60),
    'DT_EXPEDICAO': VARCHAR(20)
})

500

## Criando o tabelão com todos os dados e inserindo no banco